<a href="https://colab.research.google.com/github/dokeyuka/deep-learning-practice/blob/main/DL_practice4_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 dropout
### ランダムにdropoutするニューロンを選ぶ
##・ over fittingを防ぎモデルの汎化性能を上げる
## ・アンサンブル学習の1例 (元のmodelは一つだけど複数のモデルを生成して学習を行う)


### 注意点
#### ・学習時のみdropoutする
#### ・ニューロンの出力は1/(1-p)にスケーリングする(学習時とテスト時の出力をそろえる)
#### ・ぱらめーたの最適化に時間がかかるのでepoch数を増やす必要あり
#### ・大体p = 0.5くらいにする

In [1]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn 
import torch.optim as optimizers
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

'''
  (1) modelの実装
'''
class DNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()
    self.l1 = nn.Linear(input_dim, hidden_dim)
    self.a1 = nn.ReLU()
    self.d1 = nn.Dropout(0.5)

    self.l2 = nn.Linear(hidden_dim, hidden_dim)
    self.a2 = nn.ReLU()
    self.d2 = nn.Dropout(0.5)

    self.l3 = nn.Linear(hidden_dim,hidden_dim)
    self.a3 = nn.ReLU()
    self.d3 = nn.Dropout(0.5)

    self.l4 = nn.Linear(hidden_dim, output_dim)
    
    self.layers = [self.l1, self.a1,
                    self.l2, self.a2,
                    self.l3, self.a3,
                    self.l4]

  def forward(self,x):
    for layer in self.layers:
      x = layer(x)
    return x



if __name__ == '__main__':
  np.random.seed(123)
  #torch用の乱数シード
  torch.manual_seed(123)

  #deviceに実行環境を格納して同じコードでCPUでもGPUでも対応できるように
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
'''
  1 データの準備
'''
root = os.path.join('~', '.torch', 'mnist')
#numpyをTテンensorに変換し、さらにTensorの次元を(28，28)から(784、)に変換
transform = transforms.Compose([transforms.ToTensor(), lambda x: x.view(-1)])

mnist_train = datasets.MNIST(root = root,
                             download = True,
                             train = True,
                             transform = transform)
mnist_test = datasets.MNIST(root = root,
                            download = True,
                            train = False,
                            transform = transform)


#学習に用いるためにデータセットをDataLoaderオブジェクトに変換
#minibatch学習の時にバッチ単位でデータ処理できる、かつ、各epochでデータシャッフル可能
train_dataloader = DataLoader(mnist_train, 
                              batch_size = 100,
                              shuffle = True)

test_dataloader = DataLoader(mnist_test,
                             batch_size = 100,
                             shuffle = False)

'''
  2 モデルの構築
'''
model = DNN(784, 200, 10).to(device)

'''
  3　モデルの学習
'''
criterion = nn.CrossEntropyLoss()
optimizer = optimizers.SGD(model.parameters(), lr = 0.01)

def compute_loss(t, y):
  return criterion(y, t)

def train_step(x, t):
  #dropoutは訓練時とテスト時で挙動が異なるので書き忘れない！
  model.train()
  preds = model(x)
  loss = compute_loss(t, preds)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  return loss, preds

epochs = 30

for epoch in range(epochs):
  train_loss = 0.
  train_acc = 0.

  for(x, t) in train_dataloader:
    x,t = x.to(device), t.to(device)
    loss, preds = train_step(x,t)
    train_loss += loss.item()
    #accuracy_socreはテンソル型をうけとれないので.tolist()を実行
    train_acc += accuracy_score(t.tolist(), preds.argmax(dim = -1).tolist())
  
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)


  print('epoch:{}, loss: {:.3f}, a㏄:{:.3f}'.format(epoch +1,
                                                    train_loss,
                                                    train_acc))

'''
  4 モデルの評価
'''

def test_step(x,t):
  model.eval()
  preds = model(x)
  loss = criterion(preds, t)
  return loss, preds

test_loss = 0.
test_acc = 0.

for(x,t) in test_dataloader:
  x, t = x.to(device), t.to(device)
  loss, preds = test_step(x,t)
  test_loss += loss.item()
  test_acc += accuracy_score(t.tolist(), preds.argmax(dim = -1).tolist())
test_loss /= len(test_dataloader)
test_acc /= len(test_dataloader)


print('epoch:{}, loss: {:.3f}, acc:{:.3f}'.format(epoch +1,
                                                    test_loss,
                                                    test_acc))


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.torch/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.torch/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.torch/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.torch/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.torch/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.torch/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.torch/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.torch/mnist/MNIST/raw

epoch:1, loss: 2.273, a㏄:0.242
epoch:2, loss: 1.764, a㏄:0.617
epoch:3, loss: 0.690, a㏄:0.815
epoch:4, loss: 0.462, a㏄:0.871
epoch:5, loss: 0.391, a㏄:0.888
epoch:6, loss: 0.355, a㏄:0.898
epoch:7, loss: 0.330, a㏄:0.905
epoch:8, loss: 0.309, a㏄:0.911
epoch:9, loss: 0.291, a㏄:0.917
epoch:10, loss: 0.275, a㏄:0.921
epoch:11, loss: 0.259, a㏄:0.926
epoch:12, loss: 0.245, a㏄:0.930
epoch:13, loss: 0.232, a㏄:0.934
epoch:14, loss: 0.220, a㏄:0.938
epoch:15, loss: 0.208, a㏄:0.940
epoch:16, loss: 0.198, a㏄:0.943
epoch:17, loss: 0.188, a㏄:0.946
epoch:18, loss: 0.179, a㏄:0.949
epoch:19, loss: 0.170, a㏄:0.951
epoch:20, loss: 0.162, a㏄:0.953
epoch:21, loss: 0.155, a㏄:0.955
epoch:22, loss: 0.148, a㏄:0.958
epoch:23, loss: 0.142, a㏄:0.959
epoch:24, loss: 0.136, a㏄:0.961
epoch:25, loss: 0.130, a㏄:0.962
epoch:26, loss: 0.124, a㏄:0.964
epoch:27, loss: 0.119, a㏄:0.965
epoch:28, loss: 0.115, a㏄:0.967
epoch:29, loss